## Análisis de regionalismos

Acá vamos a crear un .csv para chequear regionalismos que detectamos previamente con un diccionario


In [10]:
%load_ext autoreload
%autoreload 2
from glob import glob
import pandas as pd

regionalism_csv = []
for csv_path in glob("../data/regionalisms/test_*.csv"):
    regionalism_csv.append(pd.read_csv(csv_path, index_col=0))

df_regionalism = pd.concat(regionalism_csv)

# Drop columns except title, text, context_tweet, prompt, pred_cot
df_regionalism = df_regionalism[["title", "text", "context_tweet", "prompt", "pred_cot"]].copy()

df_regionalism.sample(10)

,title,text,context_tweet,prompt,pred_cot
id,,,,,
349719,"Graciana Peñafort: ""La marcha me dio mucha pena y tuvo un nivel de convocatoria menor al esperado""",@usuario 🤣🤣🤣,Graciana Peñafort: “La marcha me dio mucha pena y tuvo un nivel de convocatoria menor al esperado” https://t.co/sPiBqGmFIg,"Explicar si existe un término o expresión regional del español de Argentina en el texto a continuación. Un regionalismo es una expresión de carácter usualmente coloquial usada en alguna variedad dialectal del español. No consideramos nombres propios (personas, lugares, etc) como tales. La expresión regional puede estar en el contexto (titular de noticia) o en el comentario. Razonar paso a paso antes de dar la respuesta.\n###\ncontexto: ¿More Rial cuestionó las habilidades de su ex en la cama?Luego de su separación con Facundo Ambrosioni, la mediática hizo una fuerte referencia a su vida sexual\nrespuesta: La expresión ""acumulando mas leche"" es de carácter regional, y se refiere a la acumulación de deseos sexuales. La respuesta final es sí.\n###\ncontexto: Enrique Pinti no descarta una nueva postulación de Macri: “En este país siempre hay una chance para cualquiera que nos vuelva a cagar”\nrespuesta: La expresión ""trolo"" es un término regional en Argentina usado para referirse a una...","La expresión ""🤣🤣🤣"" es un emoji que no constituye un término regional del español de Argentina. La respuesta final es no."
336555,Detuvieron a otro delincuente recientemente liberado por la excusa del coronavirus: volvió a golpear a su familia,@usuario ¿Ya echaron al juez con juicio político?,Detuvieron a un delincuente recientemente liberado por la excusa del coronavirus: volvió a golpear a su familia https://t.co/kRHjqDIljH,"Explicar si existe un término o expresión regional del español de Argentina en el texto a continuación. Un regionalismo es una expresión de carácter usualmente coloquial usada en alguna variedad dialectal del español. No consideramos nombres propios (personas, lugares, etc) como tales. La expresión regional puede estar en el contexto (titular de noticia) o en el comentario. Razonar paso a paso antes de dar la respuesta.\n###\ncontexto: ¿More Rial cuestionó las habilidades de su ex en la cama?Luego de su separación con Facundo Ambrosioni, la mediática hizo una fuerte referencia a su vida sexual\nrespuesta: La expresión ""acumulando mas leche"" es de carácter regional, y se refiere a la acumulación de deseos sexuales. La respuesta final es sí.\n###\ncontexto: Enrique Pinti no descarta una nueva postulación de Macri: “En este país siempre hay una chance para cualquiera que nos vuelva a cagar”\nrespuesta: La expresión ""trolo"" es un término regional en Argentina usado para referirse a una...","La expresión ""Ya echaron al juez con juicio político"" no contiene expresiones regionales. La respuesta final es no."
342464,"Coronavirus: Ginés Gonzalez García se comparó con China y Europa y aseguró que ""hoy se habla del modelo argentino""","@usuario @usuario por favor los compañeros taxistas se están muriendo de hambre.Te ruego AYUDA.Están la gran mayoría en categoría C, D y E. No trabajan no entra plata no hay para comprar comida ni para darle de comer a los hijos.Estamos todos desesperados. Y son los q mas aportan al País","Coronavirus: Ginés Gonzalez García se comparó con China y Europa y aseguró que ""hoy se habla del modelo argentino"" https://t.co/jQUyxLSN5X","Explicar si existe un término o expresión regional del español de Argentina en el texto a continuación. Un regionalismo es una expresión de carácter usualmente coloquial usada en alguna variedad dialectal del español. No consideramos nombres propios (personas, lugares, etc) como tales. La expresión regional puede estar en el contexto (titular de noticia) o en el comentario. Razonar paso a paso antes de dar la respuesta.\n###\ncontexto: ¿More Rial cuestionó las habilidades de su ex en la cama?Luego de su separación con Facundo Ambrosioni, la mediática hizo una fuerte refere

In [19]:
from rioplatense_hs.preprocessing import remove_accents
from rioplatense_hs.regionalisms import find_slangs
import re
from tqdm import tqdm

# Regex para buscar todas las palabras entre comillas

def find_between_quotes(text):
    return re.findall(r'"([^"]*)"', text)


def get_regionalisms(row):
    text = remove_accents(row["pred_cot"].lower())

    # Si dice "la respuesta final es si", buscar todas las ocurrencias entre comillas
    if "respuesta final es si" in text:
        return find_between_quotes(text)
    else:
        return []

tqdm.pandas()

df_regionalism["gpt_regionalisms"] = df_regionalism.apply(get_regionalisms, axis=1)
df_regionalism["dict_regionalisms"] = df_regionalism.progress_apply(lambda x: find_slangs(x["text"]), axis=1)

100%|██████████| 5670/5670 [15:16<00:00,  6.18it/s]


In [23]:
del df_regionalism["prompt"]

In [24]:
df_regionalism.to_csv("../data/regionalisms/df_regionalism.csv")

¿Cuántos marcó diccionario y cuantos gpt?

In [28]:
(df_regionalism["gpt_regionalisms"].apply(len) > 0).value_counts()

gpt_regionalisms
False    3058
True     2612
Name: count, dtype: int64

In [29]:
(df_regionalism["dict_regionalisms"].apply(len) > 0).value_counts()

dict_regionalisms
False    3375
True     2295
Name: count, dtype: int64

In [30]:
df_regionalism["has_regionalisms"] = (df_regionalism["gpt_regionalisms"].apply(len) > 0) | (df_regionalism["dict_regionalisms"].apply(len) > 0)

df_regionalism["has_regionalisms"].value_counts()


has_regionalisms
True     3412
False    2258
Name: count, dtype: int64

In [31]:
df_regionalism[df_regionalism["has_regionalisms"]].to_csv("../../data/regionalisms/check_regionalisms.csv")